# Supply chain experiments

In [18]:
import ipyleaflet as ipyl
import geopandas as gpd
import shapely.wkb 
import requests
import urllib
import os
import ee
ee.Initialize()

## Utils
**df_from_carto**

In [2]:
def df_from_carto(account, query):
    """
    It gets data by querying a carto table and converts it into a GeoDataFrame.
    """
    urlCarto = f"https://{account}.carto.com/api/v2/sql"
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns='the_geom', inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

## Basemaps

In [3]:
Mongabay_Paper = {
    "url": 'https://api.mapbox.com/styles/v1/mongabay/ckai1bze50hmj1ipohkupl0qo/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoibW9uZ2FiYXkiLCJhIjoiY2s4N25oZG82MGFyejNsb2lnN3ZrbG1jbyJ9.coW3do99wi_PmnW6OylFbA',
    "name": 'Mongabay_Paper'
}

topography_url = 'https://api.mapbox.com/styles/v1/casius/ck801p48x1hsd1iqwe67w1lac/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiY2FzaXVzIiwiYSI6ImJDMkpucTQifQ.5rm4_TsT8_PH8TzOY2V3FQ'
topography_basemap = ipyl.TileLayer(
    url=topography_url,
    layers='topography',
    format='image/png',
    name='Topography',
    opacity=1
)

## Cerrado boundary

In [4]:
account = 'p2cs-sei'
query =('SELECT * FROM brazil_biomes WHERE name = \'CERRADO\'')
df = df_from_carto(account, query)

/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
df.to_file('../data/cerrado/cerrado.shp')

In [6]:
locations = []
geometry = df['geometry'].iloc[0]
for polygon in geometry:  
    x, y = polygon.exterior.coords.xy
    locations.append(list(zip(list(y), list(x))))

## Sentinel 2 Cloud Free Composite

In [7]:
def CloudMaskS2(image):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
    parsed by Nick Clinton
    """
    AerosolsBands = ['B1']
    VIBands = ['B2', 'B3', 'B4']
    RedBands = ['B5', 'B6', 'B7', 'B8A']
    NIRBands = ['B8']
    SWIRBands = ['B11', 'B12']

    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # Both flags set to zero indicates clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
            qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)

def CloudFreeCompositeS2(startDate, stopDate):
    ## Define your collection
    collection = ee.ImageCollection('COPERNICUS/S2')

    ## Filter 
    collection = collection.filterDate(startDate,stopDate)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
            .map(CloudMaskS2)

    ## Composite
    composite = collection.median()
    
    ## normDiff bands
    normDiff_band_names = ['ndvi', 'ndwi']
    for nB, normDiff_band in enumerate([['B8','B4'], ['B8','B3']]):
        image_nd = composite.normalizedDifference(normDiff_band).rename(normDiff_band_names[nB])
        composite = ee.Image.cat([composite, image_nd])
    
    return composite

## Landsat 8 Cloud Free Composite

In [64]:
def maskL8sr(image):  
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    # Get the pixel QA band.
    qa = image.select('pixel_qa');
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 5).eq(0))
    return image.updateMask(mask)

def CloudFreeCompositeL8(startDate, stopDate):
    ## Define your collection
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

    ## Filter 
    collection = collection.filterDate(startDate,stopDate).map(maskL8sr)

    ## Composite
    composite = collection.median()
    
    return composite

## SRTM Digital Elevation Data 30m

In [36]:
dataset = ee.Image('USGS/SRTMGL1_003')
elevation_30m = dataset.select('elevation')

## USGS National Elevation Dataset 10m

In [37]:
dataset = ee.Image('USGS/NED');
elevation_10m = dataset.select('elevation');

## Area of Interest

In [74]:
polygon = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -46.79351806640625,
              -12.149430892248033
            ],
            [
              -45.274658203125,
              -12.149430892248033
            ],
            [
              -45.274658203125,
              -11.372338792141125
            ],
            [
              -46.79351806640625,
              -11.372338792141125
            ],
            [
              -46.79351806640625,
              -12.149430892248033
            ]
          ]
        ]
      }
    }
  ]
}

polygon = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -105.35682678222656,
              39.98501230042296
            ],
            [
              -105.1944351196289,
              39.98501230042296
            ],
            [
              -105.1944351196289,
              40.056789440203374
            ],
            [
              -105.35682678222656,
              40.056789440203374
            ],
            [
              -105.35682678222656,
              39.98501230042296
            ]
          ]
        ]
      }
    }
  ]
}

polygon = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -105.34412384033203,
              39.95554342883535
            ],
            [
              -105.2493667602539,
              39.95554342883535
            ],
            [
              -105.2493667602539,
              40.065723429743045
            ],
            [
              -105.34412384033203,
              40.065723429743045
            ],
            [
              -105.34412384033203,
              39.95554342883535
            ]
          ]
        ]
      }
    }
  ]
}

region = polygon.get('features')[0].get('geometry').get('coordinates')
roi = ee.Geometry.Polygon(region)

## Create Composites

In [75]:
s2_composite = CloudFreeCompositeS2(startDate='2019-01-01', stopDate='2019-12-31')
s2_composite = s2_composite.clip(roi)

viz = {'bands': ['B8', 'B4', 'B3'], 'max':3000, 'gamma': 1}
s2_NIR = s2_composite.visualize(**viz)

viz = {'bands': ['B4', 'B3', 'B2'], 'max':3000, 'gamma': 1}
s2_RGB = s2_composite.visualize(**viz)

viz = {'min':-0.75,'max':0.75, 'bands':['ndvi']}#, 'palette': ['#306466','#9cab68', '#cccc66', '#9c8448', '#6e462c']}
s2_NDVI = composite.visualize(**viz)

viz = {'min':-0.75,'max':0.75, 'bands':['ndwi']}#, 'palette': ['#8bc4f9','#c9995c', '#c7d270', '#8add60', '#097210']}
s2_NDWI = composite.visualize(**viz)

L8_composite = CloudFreeCompositeL8(startDate='2019-01-01', stopDate='2019-12-31')
L8_composite = L8_composite.clip(roi)

viz = {'min':0,'max':3000, 'gamma':1.4, 'bands':['B4', 'B3', 'B2']}
l8_RGB = L8_composite.visualize(**viz)

viz =  {'min': 1250, 'max': 2250, 'gamma': 1.6} #{'min': 0, 'max': 1000}
elevation_30m_image = elevation_30m.clip(roi).visualize(**viz)

viz =  {'min': 1250, 'max': 2250, 'gamma': 1.6} #{'min': 0, 'max': 1000}
elevation_10m_image = elevation_10m.clip(roi).visualize(**viz)

## Map

In [76]:
images = {'Elevation 30 m': elevation_30m_image,
          'Elevation 10 m': elevation_10m_image,
          'S2_RGB': s2_RGB,
          'L8_RGB': l8_RGB}

ee_tiles = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}' 

m = ipyl.Map(layers=(ipyl.basemap_to_tiles(Mongabay_Paper),),center = (roi.centroid().getInfo().get('coordinates')[1], roi.centroid().getInfo().get('coordinates')[0]), zoom=12)

m.add_layer(topography_basemap)

for name, image in images.items():
    mapid = image.getMapId()
    tiles_url = ee_tiles.format(**mapid)
    
    tile_image = ipyl.TileLayer(
        url=tiles_url,
        format='image/png',
        name=name,
        opacity=1
    )
    
    m.add_layer(tile_image)

for n, location in enumerate(locations):
    polygon = ipyl.Polygon(locations=location, color="black", weight=2, fill=False, name='Countries')
    m.add_layer(polygon)

control = ipyl.LayersControl(position='topright')
m.add_control(control)
m.add_control(ipyl.FullScreenControl())
m

Map(center=[40.010628254521095, -105.29674530029338], controls=(ZoomControl(options=['position', 'zoom_in_text…

**Export images to drive**

In [ ]:
images = {'Elevation_10m': elevation_10m_image,
          'S2_RGB': s2_RGB,}

for name, image in images.items():
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = f'export_{name}_to_Drive', 
        folder='Mongabay',
        fileNamePrefix = name,
        scale = 10, 
        region = roi.getInfo().get('coordinates'),
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF'
    )
    task.start()

**Export images as `png`**

In [77]:
visSave = {'dimensions': 1024, 'format': 'png', 'crs': 'EPSG:3857', 'region':region}
path = '../data/'

images = {'Elevation_30m_2': elevation_30m_image,
          'Elevation_10m_2': elevation_10m_image,
          'S2_RGB_2': s2_RGB,
          'L8_RGB_2': l8_RGB}

for name, image in images.items():
    url = image.getThumbURL(visSave)
    
    if os.path.exists(path+f'{name}.png'):
        os.remove(path+f'{name}.png')
    
    urllib.request.urlretrieve(url, path+f'{name}.png')